In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from random import randint
import time
from unidecode import unidecode
import numpy as np

In [15]:
df = pd.read_excel('GamesSchedule.xlsx')
df2 = pd.read_excel('Playerstats 2021-2022.xlsx') # Same as output.xlsx after some datacleaning

# Create a new DataFrame to store the selected rows
new_df = pd.DataFrame(columns=list(df2.columns) + ['Day', 'Month', 'Year', 'Team'])

In [17]:
df.head(112)

,Day,Month,Year,Westrijd,Home,Away,Uitslag
0,5,8,2022,1,sc-heerenveen,sparta-rotterdam,Draw
1,6,8,2022,2,fortuna-sittard,ajax,Loss
2,6,8,2022,3,psv,fc-emmen,Win
3,6,8,2022,4,rkc-waalwijk,fc-utrecht,Draw
4,6,8,2022,5,sc-cambuur,excelsior,Loss
...,...,...,...,...,...,...,...
107,5,11,2022,109,excelsior,sc-heerenveen,Loss
108,5,11,2022,110,fortuna-sittard,fc-emmen,Loss
109,6,11,2022,111,ajax,psv,Loss
110,6,11,2022,112,fc-twente,go-ahead-eagles,Draw


In [3]:
def get_lineups(year,month,day,home,away):
    global new_df

    if len(str(day)) < 2:
        day = "0"+str(day)
    else:
        pass 

    if len(str(month)) < 2:
        month = "0"+str(month)
    else:
        pass  

    url = f"https://www.vi.nl/wedstrijden/{year}/{month}/{day}/nederland/eredivisie/{home}-vs-{away}"
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    teams = ["home","away"]

    home_complete = False
    away_complete = False
    home_added = False

    home_lineup = []
    away_lineup = []


    for team in teams:
        div = soup.find('div', class_=f'c-pitch__team c-pitch__team--{team}')
        player_list = div.find('ul', class_='c-pitch__players-list')

        if team == "home":
            player_divs = player_list.find_all('li', class_='c-pitch__player')
            for player_div in player_divs:
                player_name = player_div.find('a').text.strip()
                player_name = player_name.strip().split(" ")[-1]
                player_name = unidecode(player_name).lower()
                home_lineup.append(unidecode(player_name))

        else:
            player_divs = player_list.find_all('li', class_='c-pitch__player')
            for player_div in player_divs:
                player_name = player_div.find('a').text.strip()
                player_name = player_name.strip().split(" ")[-1]
                player_name = unidecode(player_name).lower()
                away_lineup.append(unidecode(player_name))

        if len(home_lineup) == 11:
            home_lineup.pop(0)
            #print(len(home_lineup))
            home_complete = True

        if len(away_lineup) == 11:
            away_lineup.pop(0)
            #print(len(away_lineup))
            away_complete = True


        if home_complete == True and home_added == False:
            home_added = True
            #print("HOME COMPLETE")
            #print(len(home_lineup))
            for player in home_lineup:
            # Check if player is present in any Slug
                is_present = df2['Slug'].str.contains(player)
            
                # Count the occurrences of player in Slug
                name_count = is_present.sum()
                if name_count > 1:
                    #print(f"There are {name_count} {player}s")
                    # Add NaN rows with player name, day, month, year, and team for the player in the new DataFrame 
                    nan_row = pd.Series([np.nan] * len(new_df.columns), index=new_df.columns)
                    nan_row['Slug'] = player
                    nan_row['Day'] = day
                    nan_row['Month'] = month
                    nan_row['Year'] = year
                    nan_row['Team'] = team
                    new_df = pd.concat([new_df, pd.DataFrame([nan_row])], ignore_index=True)
            
                if is_present.any() and name_count == 1:
                    #print(player, "is there")
                    # Get the row where player is present
                    row_with_player = df2.loc[is_present].copy()
                    
                    # Add the specified day, month, year, and team to the row using .loc accessor
                    row_with_player.loc[:, 'Day'] = day
                    row_with_player.loc[:, 'Month'] = month
                    row_with_player.loc[:, 'Year'] = year
                    row_with_player.loc[:, 'Team'] = team
                    
                    new_df = pd.concat([new_df, row_with_player])
    
        if away_complete == True:
            for player in away_lineup:
            # Check if player is present in any Slug
                is_present = df2['Slug'].str.contains(player)
            
                # Count the occurrences of player in Slug
                name_count = is_present.sum()
                if name_count > 1:
                    #print(f"There are {name_count} {player}s")
                    # Add NaN rows with player name, day, month, year, and team for the player in the new DataFrame 
                    nan_row = pd.Series([np.nan] * len(new_df.columns), index=new_df.columns)
                    nan_row['Slug'] = player
                    nan_row['Day'] = day
                    nan_row['Month'] = month
                    nan_row['Year'] = year
                    nan_row['Team'] = team
                    new_df = pd.concat([new_df, pd.DataFrame([nan_row])], ignore_index=True)
            
                if is_present.any() and name_count == 1:
                    #print(player, "is there")
                    # Get the row where player is present
                    row_with_player = df2.loc[is_present].copy()
                    
                    # Add the specified day, month, year, and team to the row using .loc accessor
                    row_with_player.loc[:, 'Day'] = day
                    row_with_player.loc[:, 'Month'] = month
                    row_with_player.loc[:, 'Year'] = year
                    row_with_player.loc[:, 'Team'] = team
                    
                    new_df = pd.concat([new_df, row_with_player])
                    
            

In [20]:
df = pd.read_excel('GamesSchedule.xlsx')
df2 = pd.read_excel('Playerstats 2021-2022.xlsx')

# Create a new DataFrame to store the selected rows
new_df = pd.DataFrame(columns=list(df2.columns) + ['Day', 'Month', 'Year', 'Team'])

#new_df.head()

for i in range(len(df)):
    print("----------------------------------------------------------------------------------------------------------------------")
    day = df.iloc[i][0]
    month = df.iloc[i][1]
    year = df.iloc[i][2]
    home = df.iloc[i][4]
    away = df.iloc[i][5]

    print(year,month,day,home,away)
    
    interval = randint(1,5)
    print("Waiting for",interval," seconds.")
    time.sleep(interval) # Delay to not spam website API

    get_lineups(year,month,day,home,away)


new_df.head()

#CRASHED AT VITESSE VS SPARTA ROTTERDAM 4-11-2022

----------------------------------------------------------------------------------------------------------------------
2022 8 5 sc-heerenveen sparta-rotterdam
Waiting for 3  seconds.
https://www.vi.nl/wedstrijden/2022/08/05/nederland/eredivisie/sc-heerenveen-vs-sparta-rotterdam
----------------------------------------------------------------------------------------------------------------------
2022 8 6 fortuna-sittard ajax
Waiting for 2  seconds.
https://www.vi.nl/wedstrijden/2022/08/06/nederland/eredivisie/fortuna-sittard-vs-ajax
----------------------------------------------------------------------------------------------------------------------
2022 8 6 psv fc-emmen
Waiting for 1  seconds.
https://www.vi.nl/wedstrijden/2022/08/06/nederland/eredivisie/psv-vs-fc-emmen
----------------------------------------------------------------------------------------------------------------------
2022 8 6 rkc-waalwijk fc-utrecht
Waiting for 2  seconds.
https://www.vi.nl/wedstrijden/2022/08/06/ne

,Slug,WG,WI,MG,G,AST,GK,RK,SH,SH%,...,TCK,TCK%,INT,BVL,BVE,FOUT,Day,Month,Year,Team
0,joost-van-wang-aken,3,1,219,1,0,0,0,5,2,...,6,4,6,11,3,1,05,08,2022,home
1,sven-van-beek,31,0,2938,1,0,4,0,19,8,...,107,75,280,119,37,8,05,08,2022,home
2,rami-kaib,25,1,2164,0,1,5,0,24,6,...,80,37,120,190,65,5,05,08,2022,home
3,milan-van-ewijk,35,0,3285,1,1,5,0,14,5,...,152,116,261,215,74,9,05,08,2022,home
4,mats-kohlert,31,4,2425,3,2,1,0,26,11,...,101,56,110,196,37,4,05,08,2022,home


In [21]:
new_df.tail()

,Slug,WG,WI,MG,G,AST,GK,RK,SH,SH%,...,TCK,TCK%,INT,BVL,BVE,FOUT,Day,Month,Year,Team
82,jamal-amofa,6,1,485,0,0,1,0,0,0,...,22,10,29,18,7,4,28,05,2023,away
130,mats-deijl,31,4,2506,2,2,6,0,13,7,...,114,65,152,194,53,4,28,05,2023,away
185,philippe-rommens,33,3,2819,2,6,2,0,23,7,...,111,56,157,190,46,3,28,05,2023,away
25,bobby-adekanye,14,3,775,2,0,1,0,12,6,...,17,10,16,74,16,2,28,05,2023,away
78,isac-lidberg,31,7,1988,4,2,3,1,38,17,...,31,14,42,273,49,1,28,05,2023,away


In [22]:
new_df.to_excel('lineups.xlsx', index=False)